In [2]:
import csv
import pandas as pd
import numpy as np

In [3]:
# Read MIMICs CSV file
mimic_mean_df = pd.read_csv('..\\01_MimicIV\\CSV\\Exports\\datasets\\whole_set\\o01_final_mean_with_los.csv', low_memory=False)
mimic_median_df = pd.read_csv('..\\01_MimicIV\\CSV\\Exports\\datasets\\whole_set\\o02_final_median_with_los.csv', low_memory=False)
mimic_min_df = pd.read_csv('..\\01_MimicIV\\CSV\\Exports\\datasets\\whole_set\\o03_final_min_with_los.csv', low_memory=False)
mimic_max_df = pd.read_csv('..\\01_MimicIV\\CSV\\Exports\\datasets\\whole_set\\o04_final_max_with_los.csv', low_memory=False)

# Read eICUs CSV file
eicu_meam_df = pd.read_csv('..\\02_eICU\\CSV\\Exports\\datasets\\whole_set\\o01_final_mean_table.csv', low_memory=False)
eicu_median_df = pd.read_csv('..\\02_eICU\\CSV\\Exports\\datasets\\whole_set\\o02_final_median_table.csv', low_memory=False)
eicu_min_df = pd.read_csv('..\\02_eICU\\CSV\\Exports\\datasets\\whole_set\\o03_final_min_table.csv', low_memory=False)
eicu_max_df = pd.read_csv('..\\02_eICU\\CSV\\Exports\\datasets\\whole_set\\o04_final_max_table.csv', low_memory=False)

In [22]:
# columns to keep
mimic_columns_to_keep = pd.read_csv('CSV\\imports\\mimic_features.csv')
eicu_columns_to_keep = pd.read_csv("CSV\\imports\\eicu_features.csv")

# MIMIC

In [6]:
# Merge mimics dataframes
merged_mimic_df = mimic_mean_df.merge(mimic_median_df, on=['row_count','subject_id', 'hadm_id', 'Time_Zone', 'gender', 'age', 'language', 'marital_status', 'race', 'hospital_expire_flag', 'los'], suffixes=('_mean', '_median'))
merged_mimic_df = merged_mimic_df.merge(mimic_min_df, on=['row_count','subject_id', 'hadm_id', 'Time_Zone', 'gender', 'age', 'language', 'marital_status', 'race', 'hospital_expire_flag', 'los'], suffixes=('', '_min'))
merged_mimic_df = merged_mimic_df.merge(mimic_max_df, on=['row_count','subject_id', 'hadm_id', 'Time_Zone', 'gender', 'age', 'language', 'marital_status', 'race', 'hospital_expire_flag', 'los'], suffixes=('', '_max'))

# Rename columns to replace suffixes
merged_mimic_df.columns = merged_mimic_df.columns.str.replace(r'\s*-\s*Mean', ' (Mean)', regex=True)
merged_mimic_df.columns = merged_mimic_df.columns.str.replace(r'\s*-\s*Median', ' (Median)', regex=True)
merged_mimic_df.columns = merged_mimic_df.columns.str.replace(r'\s*-\s*Min', ' (Min)', regex=True)
merged_mimic_df.columns = merged_mimic_df.columns.str.replace(r'\s*-\s*Max', ' (Max)', regex=True)

# Move the 'hospital_expire_flag' and 'LOS' columns to the end of the dataframe
hospital_expire_flag_column = merged_mimic_df.pop('hospital_expire_flag')
los_column = merged_mimic_df.pop('los')
merged_mimic_df = pd.concat([merged_mimic_df, hospital_expire_flag_column, los_column], axis=1)

# Rename the last two columns to preserve their original names
merged_mimic_df.columns = list(merged_mimic_df.columns[:-2]) + ['hospital_expire_flag', 'los']

In [7]:
# Summarize GCS components for Mean, Median, Min, and Max while handling NaNs
merged_mimic_df['GCS (Mean)'] = merged_mimic_df.apply(lambda row: row[['GCS - Eye Opening (Mean)', 'GCS - Verbal Response (Mean)', 'GCS - Motor Response (Mean)']].sum() if not all(row[['GCS - Eye Opening (Mean)', 'GCS - Verbal Response (Mean)', 'GCS - Motor Response (Mean)']].isna()) else np.nan, axis=1)
merged_mimic_df['GCS (Median)'] = merged_mimic_df.apply(lambda row: row[['GCS - Eye Opening (Median)', 'GCS - Verbal Response (Median)', 'GCS - Motor Response (Median)']].sum() if not all(row[['GCS - Eye Opening (Median)', 'GCS - Verbal Response (Median)', 'GCS - Motor Response (Median)']].isna()) else np.nan, axis=1)
merged_mimic_df['GCS (Min)'] = merged_mimic_df.apply(lambda row: row[['GCS - Eye Opening (Min)', 'GCS - Verbal Response (Min)', 'GCS - Motor Response (Min)']].sum() if not all(row[['GCS - Eye Opening (Min)', 'GCS - Verbal Response (Min)', 'GCS - Motor Response (Min)']].isna()) else np.nan, axis=1)
merged_mimic_df['GCS (Max)'] = merged_mimic_df.apply(lambda row: row[['GCS - Eye Opening (Max)', 'GCS - Verbal Response (Max)', 'GCS - Motor Response (Max)']].sum() if not all(row[['GCS - Eye Opening (Max)', 'GCS - Verbal Response (Max)', 'GCS - Motor Response (Max)']].isna()) else np.nan, axis=1)

# Drop the original GCS component columns
merged_mimic_df.drop(columns=[
    'GCS - Eye Opening (Mean)', 'GCS - Verbal Response (Mean)', 'GCS - Motor Response (Mean)',
    'GCS - Eye Opening (Median)', 'GCS - Verbal Response (Median)', 'GCS - Motor Response (Median)',
    'GCS - Eye Opening (Min)', 'GCS - Verbal Response (Min)', 'GCS - Motor Response (Min)',
    'GCS - Eye Opening (Max)', 'GCS - Verbal Response (Max)', 'GCS - Motor Response (Max)'
], inplace=True)

In [8]:
# Summarize Braden components for Mean, Median, Min, and Max while handling NaNs
merged_mimic_df['Braden (Mean)'] = merged_mimic_df.apply(lambda row: row[['Braden Sensory Perception (Mean)', 'Braden Moisture (Mean)', 'Braden Activity (Mean)', 'Braden Mobility (Mean)', 'Braden Nutrition (Mean)', 'Braden Friction/Shear (Mean)']].sum() if not all(row[['Braden Sensory Perception (Mean)', 'Braden Moisture (Mean)', 'Braden Activity (Mean)', 'Braden Mobility (Mean)', 'Braden Nutrition (Mean)', 'Braden Friction/Shear (Mean)']].isna()) else np.nan, axis=1)
merged_mimic_df['Braden (Median)'] = merged_mimic_df.apply(lambda row: row[['Braden Sensory Perception (Median)', 'Braden Moisture (Median)', 'Braden Activity (Median)', 'Braden Mobility (Median)', 'Braden Nutrition (Median)', 'Braden Friction/Shear (Median)']].sum() if not all(row[['Braden Sensory Perception (Median)', 'Braden Moisture (Median)', 'Braden Activity (Median)', 'Braden Mobility (Median)', 'Braden Nutrition (Median)', 'Braden Friction/Shear (Median)']].isna()) else np.nan, axis=1)
merged_mimic_df['Braden (Min)'] = merged_mimic_df.apply(lambda row: row[['Braden Sensory Perception (Min)', 'Braden Moisture (Min)', 'Braden Activity (Min)', 'Braden Mobility (Min)', 'Braden Nutrition (Min)', 'Braden Friction/Shear (Min)']].sum() if not all(row[['Braden Sensory Perception (Min)', 'Braden Moisture (Min)', 'Braden Activity (Min)', 'Braden Mobility (Min)', 'Braden Nutrition (Min)', 'Braden Friction/Shear (Min)']].isna()) else np.nan, axis=1)
merged_mimic_df['Braden (Max)'] = merged_mimic_df.apply(lambda row: row[['Braden Sensory Perception (Max)', 'Braden Moisture (Max)', 'Braden Activity (Max)', 'Braden Mobility (Max)', 'Braden Nutrition (Max)', 'Braden Friction/Shear (Max)']].sum() if not all(row[['Braden Sensory Perception (Max)', 'Braden Moisture (Max)', 'Braden Activity (Max)', 'Braden Mobility (Max)', 'Braden Nutrition (Max)', 'Braden Friction/Shear (Max)']].isna()) else np.nan, axis=1)

# Drop the original Braden component columns
merged_mimic_df.drop(columns=[
    'Braden Sensory Perception (Mean)', 'Braden Moisture (Mean)', 'Braden Activity (Mean)', 'Braden Mobility (Mean)', 'Braden Nutrition (Mean)', 'Braden Friction/Shear (Mean)',
    'Braden Sensory Perception (Median)', 'Braden Moisture (Median)', 'Braden Activity (Median)', 'Braden Mobility (Median)', 'Braden Nutrition (Median)', 'Braden Friction/Shear (Median)',
    'Braden Sensory Perception (Min)', 'Braden Moisture (Min)', 'Braden Activity (Min)', 'Braden Mobility (Min)', 'Braden Nutrition (Min)', 'Braden Friction/Shear (Min)',
    'Braden Sensory Perception (Max)', 'Braden Moisture (Max)', 'Braden Activity (Max)', 'Braden Mobility (Max)', 'Braden Nutrition (Max)', 'Braden Friction/Shear (Max)'
], inplace=True)

In [9]:
# Remove spaces and commas
merged_mimic_df.columns = merged_mimic_df.columns.str.replace(r'[ ,]+', '_', regex=True)

In [10]:
# Drop second column from the column_names_df
mimic_columns_to_keep.drop(columns=['Unnamed: 1'], inplace=True)

# Extract column names from columns_to_keep DataFrame
columns_to_keep_names = mimic_columns_to_keep['column'].tolist()

# Select only the desired columns
mimic_temp = merged_mimic_df[columns_to_keep_names]

In [11]:
# Remove Duplicate Columns
df_mimic_unique = mimic_temp.loc[:, ~mimic_temp.columns.duplicated()]

In [12]:
# Multiply values by 4 in 'Ionized Calcium' column, leaving NaN values unchanged for normalization with eicu
df_mimic_unique.loc[:, 'Ionized_Calcium_(Max)'] = mimic_temp['Ionized_Calcium_(Max)'].apply(lambda x: x * 4 if pd.notna(x) else x)
df_mimic_unique.loc[:, 'Ionized_Calcium_(Mean)'] = mimic_temp['Ionized_Calcium_(Mean)'].apply(lambda x: x * 4 if pd.notna(x) else x)
df_mimic_unique.loc[:, 'Ionized_Calcium_(Median)'] = mimic_temp['Ionized_Calcium_(Median)'].apply(lambda x: x * 4 if pd.notna(x) else x)
df_mimic_unique.loc[:, 'Ionized_Calcium_(Min)'] = mimic_temp['Ionized_Calcium_(Min)'].apply(lambda x: x * 4 if pd.notna(x) else x)

In [13]:
# Glucose merge
df_mimic_unique.loc[:, 'Glucose (Max)'] = df_mimic_unique.apply(lambda row: row[['Glucose_(Max)', 'Glucose_(Max).1', 'Glucose_(Max).2']].mean() if not all(row[['Glucose_(Max)', 'Glucose_(Max).1', 'Glucose_(Max).2']].isna()) else np.nan, axis=1)
df_mimic_unique.loc[:, 'Glucose (Mean)'] = df_mimic_unique.apply(lambda row: row[['Glucose_(Mean)', 'Glucose_(Mean).1', 'Glucose_(Mean).2']].mean() if not all(row[['Glucose_(Mean)', 'Glucose_(Mean).1', 'Glucose_(Mean).2']].isna()) else np.nan, axis=1)
df_mimic_unique.loc[:, 'Glucose (Median)'] = df_mimic_unique.apply(lambda row: row[['Glucose_(Median)', 'Glucose_(Median).1', 'Glucose_(Median).2']].mean() if not all(row[['Glucose_(Median)', 'Glucose_(Median).1', 'Glucose_(Median).2']].isna()) else np.nan, axis=1)
df_mimic_unique.loc[:, 'Glucose (Min)'] = df_mimic_unique.apply(lambda row: row[['Glucose_(Min)', 'Glucose_(Min).1', 'Glucose_(Min).2']].mean() if not all(row[['Glucose_(Min)', 'Glucose_(Min).1', 'Glucose_(Min).2']].isna()) else np.nan, axis=1)

# Drop original Glucose columns to keep only the summarized columns
df_mimic_unique.drop(columns=[
    'Glucose_(Max)', 'Glucose_(Max).1', 'Glucose_(Max).2',
    'Glucose_(Mean)', 'Glucose_(Mean).1', 'Glucose_(Mean).2',
    'Glucose_(Median)', 'Glucose_(Median).1', 'Glucose_(Median).2',
    'Glucose_(Min)', 'Glucose_(Min).1', 'Glucose_(Min).2'
], inplace=True)

C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\2871867564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mimic_unique.loc[:, 'Glucose (Max)'] = df_mimic_unique.apply(lambda row: row[['Glucose_(Max)', 'Glucose_(Max).1', 'Glucose_(Max).2']].mean() if not all(row[['Glucose_(Max)', 'Glucose_(Max).1', 'Glucose_(Max).2']].isna()) else np.nan, axis=1)
C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\2871867564.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mimic_unique.loc[:, 'Glucose

In [14]:
# pH merge
df_mimic_unique.loc[:, 'pH (Max)'] = df_mimic_unique.apply(lambda row: row[['pH_(Max)', 'pH_(Max).1', 'pH_(Max).3']].mean() if not all(row[['pH_(Max)', 'pH_(Max).1', 'pH_(Max).3']].isna()) else np.nan, axis=1)
df_mimic_unique.loc[:, 'pH (Mean)'] = df_mimic_unique.apply(lambda row: row[['pH_(Mean)', 'pH_(Mean).1', 'pH_(Mean).2', 'pH_(Mean).3']].mean() if not all(row[['pH_(Mean)', 'pH_(Mean).1', 'pH_(Mean).2', 'pH_(Mean).3']].isna()) else np.nan, axis=1)
df_mimic_unique.loc[:, 'pH (Median)'] = df_mimic_unique.apply(lambda row: row[['pH_(Median)', 'pH_(Median).1', 'pH_(Median).3']].mean() if not all(row[['pH_(Median)', 'pH_(Median).1', 'pH_(Median).3']].isna()) else np.nan, axis=1)
df_mimic_unique.loc[:, 'pH (Min)'] = df_mimic_unique.apply(lambda row: row[['pH_(Min)', 'pH_(Min).1', 'pH_(Min).3']].mean() if not all(row[['pH_(Min)', 'pH_(Min).1', 'pH_(Min).3']].isna()) else np.nan, axis=1)

# Drop original pH columns to keep only the summarized columns
df_mimic_unique.drop(columns=[
    'pH_(Max)', 'pH_(Max).1', 'pH_(Max).3',
    'pH_(Mean)', 'pH_(Mean).1', 'pH_(Mean).2', 'pH_(Mean).3',
    'pH_(Median)', 'pH_(Median).1', 'pH_(Median).3',
    'pH_(Min)', 'pH_(Min).1', 'pH_(Min).3'
], inplace=True)

C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\2439638156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mimic_unique.loc[:, 'pH (Max)'] = df_mimic_unique.apply(lambda row: row[['pH_(Max)', 'pH_(Max).1', 'pH_(Max).3']].mean() if not all(row[['pH_(Max)', 'pH_(Max).1', 'pH_(Max).3']].isna()) else np.nan, axis=1)
C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\2439638156.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mimic_unique.loc[:, 'pH (Mean)'] = df_mimic_unique.apply(lambda

# eICU

In [50]:
# Merge eICU dataframes
merged_eicu_df = eicu_meam_df.merge(eicu_median_df, on=['row_count', 'uniquepid', 'patientunitstayid', 'Time_Zone', 'gender', 'age', 'ethnicity', 'unitdischargestatus', 'LOS'], suffixes=('_mean', '_median'))
merged_eicu_df = merged_eicu_df.merge(eicu_min_df, on=['row_count', 'uniquepid', 'patientunitstayid', 'Time_Zone', 'gender', 'age', 'ethnicity', 'unitdischargestatus', 'LOS'], suffixes=('', '_min'))
merged_eicu_df = merged_eicu_df.merge(eicu_max_df, on=['row_count', 'uniquepid', 'patientunitstayid', 'Time_Zone', 'gender', 'age', 'ethnicity', 'unitdischargestatus', 'LOS'], suffixes=('', '_max'))

# Move the 'unitdischargestatus' and 'LOS' columns to the end of the dataframe
unitdischargestatus_column = merged_eicu_df.pop('unitdischargestatus')
los_column = merged_eicu_df.pop('LOS')
merged_eicu_df = pd.concat([merged_eicu_df, unitdischargestatus_column, los_column], axis=1)

# Rename the last two columns to preserve their original names
merged_eicu_df.columns = list(merged_eicu_df.columns[:-2]) + ['unitdischargestatus', 'LOS']

In [52]:
# Drop second column from the column_names_df
#eicu_columns_to_keep.drop(columns=['Unnamed: 1'], inplace=True)

# Extract column names from columns_to_keep DataFrame
columns_to_keep_names = eicu_columns_to_keep['column'].tolist()

# Select only the desired columns
eicu_temp = merged_eicu_df[columns_to_keep_names]

In [54]:
"""--------Replace Block----------"""

# Replace 'Alive' with 0 and 'Expired' with 1 in the 'unitdischargestatus' column
eicu_temp.loc[:, 'unitdischargestatus'] = eicu_temp['unitdischargestatus'].replace({'Alive': 0, 'Expired': 1})

# Replace 'Female' with 'F' and 'Male' with 'M' in the 'gender' column
eicu_temp.loc[:, 'gender'] = eicu_temp['gender'].replace({'Female': 'F', 'Male': 'M'})


# Replace values in the 'ethnicity' column for standardization
eicu_temp.loc[:, 'ethnicity'] = eicu_temp['ethnicity'].replace({
    'African American': 'BLACK/AFRICAN AMERICAN',
    'Caucasian': 'WHITE',
    'Hispanic': 'HISPANIC OR LATINO',
    'Asian': 'ASIAN',
    'Native American': 'AMERICAN INDIAN/ALASKA NATIVE',
    'Other/Unknown': 'UNKNOWN'
})

# Replace age values higher than 89 with 90, and convert age to integer
eicu_temp.loc[:, 'age'] = eicu_temp['age'].replace('> 89', 90)
eicu_temp.loc[:, 'age'] = eicu_temp['age'].astype(np.int64)

C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\1356455573.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  eicu_temp.loc[:, 'unitdischargestatus'] = eicu_temp['unitdischargestatus'].replace({'Alive': 0, 'Expired': 1})


In [56]:
# Remove spaces and commas
df_mimic_unique.columns = df_mimic_unique.columns.str.replace(r'[ ,]+', '_', regex=True)

eicu_temp.columns = eicu_temp.columns.str.replace(r'[ ,]+', '_', regex=True)

In [ ]:
# Bedside glucosd
eicu_temp.loc[:, 'bedside_glucose (Max)'] = eicu_temp.apply(
    lambda row: row[['bedside_glucose_(Max)', 'Bedside_Glucose_(Max)']].mean() if not all(row[['bedside_glucose_(Max)', 'Bedside_Glucose_(Max)']].isna()) else np.nan,
    axis=1
)
eicu_temp.loc[:, 'bedside_glucose (Mean)'] = eicu_temp.apply(
    lambda row: row[['bedside_glucose_(Mean)', 'Bedside_Glucose_(Mean)']].mean() if not all(row[['bedside_glucose_(Mean)', 'Bedside_Glucose_(Mean)']].isna()) else np.nan,
    axis=1
)
eicu_temp.loc[:, 'bedside_glucose (Median)'] = eicu_temp.apply(
    lambda row: row[['bedside_glucose_(Median)', 'Bedside_Glucose_(Median)']].mean() if not all(row[['bedside_glucose_(Median)', 'Bedside_Glucose_(Median)']].isna()) else np.nan,
    axis=1
)
eicu_temp.loc[:, 'bedside_glucose (Min)'] = eicu_temp.apply(
    lambda row: row[['bedside_glucose_(Min)', 'Bedside_Glucose_(Min)']].mean() if not all(row[['bedside_glucose_(Min)', 'Bedside_Glucose_(Min)']].isna()) else np.nan,
    axis=1
)

# Drop original bedside glucose columns to keep only the summarized ones
eicu_temp.drop(columns=[
    'bedside_glucose_(Max)', 'Bedside_Glucose_(Max)',
    'bedside_glucose_(Mean)', 'Bedside_Glucose_(Mean)',
    'bedside_glucose_(Median)', 'Bedside_Glucose_(Median)',
    'bedside_glucose_(Min)', 'Bedside_Glucose_(Min)'
], inplace=True)

eicu_temp.columns = eicu_temp.columns.str.replace(r'[ ,]+', '_', regex=True)

C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\1637545364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eicu_temp.loc[:, 'bedside_glucose (Max)'] = eicu_temp.apply(
C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\1637545364.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eicu_temp.loc[:, 'bedside_glucose (Mean)'] = eicu_temp.apply(


In [ ]:
#Export Columns
column_names = merged_eicu_df.columns

# Create a dataframe from the column names
columns_df = pd.DataFrame(column_names, columns=['Column_Name'])

# Export to CSV
columns_df.to_csv('column_names.csv', index=False)

In [ ]:
# Check specific columns and missing values

braden_columns = [col for col in merged_eicu_df.columns if col.startswith('Invasive BP Diastolic')]
braden_df = merged_eicu_df[braden_columns]

# Count the missing values for each GCS column
missing_values_count = braden_df.isnull().sum()

In [ ]:
display(missing_values_count)

In [36]:
column_eicu_mapping = {
    'column': 'column',
    'row_count': 'row_count',
    'uniquepid': 'subject_id',
    'patientunitstayid': 'hadm_id',
    'Time_Zone': 'Time_Zone',
    'gender': 'gender',
    'age': 'age',
    'ethnicity': 'race',
    'Base_Excess_(Max)': 'Base_Excess_(Max)',
    'Base_Excess_(Mean)': 'Base_Excess_(Mean)',
    'Base_Excess_(Median)': 'Base_Excess_(Median)',
    'Base_Excess_(Min)': 'Base_Excess_(Min)',
    'lactate_(Max)': 'Lactate_(Max)',
    'lactate_(Mean)': 'Lactate_(Mean)',
    'lactate_(Median)': 'Lactate_(Median)',
    'lactate_(Min)': 'Lactate_(Min)',
    'paCO2_(Max)': 'pCO2_(Max)',
    'paCO2_(Mean)': 'pCO2_(Mean)',
    'paCO2_(Median)': 'pCO2_(Median)',
    'paCO2_(Min)': 'pCO2_(Min)',
    'Total_CO2_(Max)': 'Calculated_Total_CO2_(Max)',
    'Total_CO2_(Mean)': 'Calculated_Total_CO2_(Mean)',
    'Total_CO2_(Median)': 'Calculated_Total_CO2_(Median)',
    'Total_CO2_(Min)': 'Calculated_Total_CO2_(Min)',
    'BUN_(Max)': 'BUN_(Max)',
    'BUN_(Mean)': 'BUN_(Mean)',
    'BUN_(Median)': 'BUN_(Median)',
    'BUN_(Min)': 'BUN_(Min)',
    'pH_(Max)': 'pH_(Max)',
    'pH_(Mean)': 'pH_(Mean)',
    'pH_(Median)': 'pH_(Median)',
    'pH_(Min)': 'pH_(Min)',
    'paO2_(Max)': 'pO2_(Max)',
    'paO2_(Mean)': 'pO2_(Mean)',
    'paO2_(Median)': 'pO2_(Median)',
    'paO2_(Min)': 'pO2_(Min)',
    'ALT_(SGPT)_(Max)': 'Alanine_Aminotransferase_(ALT)_(Max)',
    'ALT_(SGPT)_(Mean)': 'Alanine_Aminotransferase_(ALT)_(Mean)',
    'ALT_(SGPT)_(Median)': 'Alanine_Aminotransferase_(ALT)_(Median)',
    'ALT_(SGPT)_(Min)': 'Alanine_Aminotransferase_(ALT)_(Min)',
    'alkaline_phos._(Max)': 'Alkaline_Phosphatase_(Max)',
    'alkaline_phos._(Mean)': 'Alkaline_Phosphatase_(Mean)',
    'alkaline_phos._(Median)': 'Alkaline_Phosphatase_(Median)',
    'alkaline_phos._(Min)': 'Alkaline_Phosphatase_(Min)',
    'anion_gap_(Max)': 'Anion_Gap_(Max)',
    'anion_gap_(Mean)': 'Anion_Gap_(Mean)',
    'anion_gap_(Median)': 'Anion_Gap_(Median)',
    'anion_gap_(Min)': 'Anion_Gap_(Min)',
    'AST_(SGOT)_(Max)': 'Asparate_Aminotransferase_(AST)_(Max)',
    'AST_(SGOT)_(Mean)': 'Asparate_Aminotransferase_(AST)_(Mean)',
    'AST_(SGOT)_(Median)': 'Asparate_Aminotransferase_(AST)_(Median)',
    'AST_(SGOT)_(Min)': 'Asparate_Aminotransferase_(AST)_(Min)',
    'bicarbonate_(Max)': 'Bicarbonate_(Max)',
    'bicarbonate_(Mean)': 'Bicarbonate_(Mean)',
    'bicarbonate_(Median)': 'Bicarbonate_(Median)',
    'bicarbonate_(Min)': 'Bicarbonate_(Min)',
    'chloride_(Max)': 'Chloride_(Max)',
    'chloride_(Mean)': 'Chloride_(Mean)',
    'chloride_(Median)': 'Chloride_(Median)',
    'chloride_(Min)': 'Chloride_(Min)',
    'creatinine_(Max)': 'Creatinine_(Max)',
    'creatinine_(Mean)': 'Creatinine_(Mean)',
    'creatinine_(Median)': 'Creatinine_(Median)',
    'creatinine_(Min)': 'Creatinine_(Min)',
    'glucose_(Max)': 'Glucose_(Max)',
    'glucose_(Mean)': 'Glucose_(Mean)',
    'glucose_(Median)': 'Glucose_(Median)',
    'glucose_(Min)': 'Glucose_(Min)',
    'magnesium_(Max)': 'Magnesium_(Max)',
    'magnesium_(Mean)': 'Magnesium_(Mean)',
    'magnesium_(Median)': 'Magnesium_(Median)',
    'magnesium_(Min)': 'Magnesium_(Min)',
    'phosphate_(Max)': 'Phosphate_(Max)',
    'phosphate_(Mean)': 'Phosphate_(Mean)',
    'phosphate_(Median)': 'Phosphate_(Median)',
    'phosphate_(Min)': 'Phosphate_(Min)',
    'potassium_(Max)': 'Potassium_(Max)',
    'potassium_(Mean)': 'Potassium_(Mean)',
    'potassium_(Median)': 'Potassium_(Median)',
    'potassium_(Min)': 'Potassium_(Min)',
    'sodium_(Max)': 'Sodium_(Max)',
    'sodium_(Mean)': 'Sodium_(Mean)',
    'sodium_(Median)': 'Sodium_(Median)',
    'sodium_(Min)': 'Sodium_(Min)',
    'Hct_(Max)': 'Hematocrit_(Max)',
    'Hct_(Mean)': 'Hematocrit_(Mean)',
    'Hct_(Median)': 'Hematocrit_(Median)',
    'Hct_(Min)': 'Hematocrit_(Min)',
    'Hgb_(Max)': 'Hemoglobin_(Max)',
    'Hgb_(Mean)': 'Hemoglobin_(Mean)',
    'Hgb_(Median)': 'Hemoglobin_(Median)',
    'Hgb_(Min)': 'Hemoglobin_(Min)',
    'PT_-_INR_(Max)': 'INR(PT)_(Max)',
    'PT_-_INR_(Mean)': 'INR(PT)_(Mean)',
    'PT_-_INR_(Median)': 'INR(PT)_(Median)',
    'PPT_-_INR_(Min)': 'INR(PT)_(Min)',
    'MCH_(Max)': 'MCH_(Max)',
    'MCH_(Mean)': 'MCH_(Mean)',
    'MCH_(Median)': 'MCH_(Median)',
    'MCH_(Min)': 'MCH_(Min)',
    'MCHC_(Max)': 'MCHC_(Max)',
    'MCHC_(Mean)': 'MCHC_(Mean)',
    'MCHC_(Median)': 'MCHC_(Median)',
    'MCHC_(Min)': 'MCHC_(Min)',
    'MCV_(Max)': 'MCV_(Max)',
    'MCV_(Mean)': 'MCV_(Mean)',
    'MCV_(Median)': 'MCV_(Median)',
    'MCV_(Min)': 'MCV_(Min)',
    'platelets_x_1000_(Max)': 'Platelet_Count_(Max)',
    'platelets_x_1000_(Mean)': 'Platelet_Count_(Mean)',
    'platelets_x_1000_(Median)': 'Platelet_Count_(Median)',
    'platelets_x_1000_(Min)': 'Platelet_Count_(Min)',
    'PT_(Max)': 'PT_(Max)',
    'PT_(Mean)': 'PT_(Mean)',
    'PT_(Median)': 'PT_(Median)',
    'PT_(Min)': 'PT_(Min)',
    'PTT_(Max)': 'PTT_(Max)',
    'PTT_(Mean)': 'PTT_(Mean)',
    'PTT_(Median)': 'PTT_(Median)',
    'PTT_(Min)': 'PTT_(Min)',
    'RDW_(Max)': 'RDW_(Max)',
    'RDW_(Mean)': 'RDW_(Mean)',
    'RDW_(Median)': 'RDW_(Median)',
    'RDW_(Min)': 'RDW_(Min)',
    'RBC_(Max)': 'Red_Blood_Cells_(Max)',
    'RBC_(Mean)': 'Red_Blood_Cells_(Mean)',
    'RBC_(Median)': 'Red_Blood_Cells_(Median)',
    'RBC_(Min)': 'Red_Blood_Cells_(Min)',
    'WBC_x_1000_(Max)': 'White_Blood_Cells_(Max)',
    'WBC_x_1000_(Mean)': 'White_Blood_Cells_(Mean)',
    'WBC_x_1000_(Median)': 'White_Blood_Cells_(Median)',
    'WBC_x_1000_(Min)': 'White_Blood_Cells_(Min)',
    'Heart_Rate_(Max)': 'Heart_Rate_(bpm)_(Max)',
    'Heart_Rate_(Mean)': 'Heart_Rate_(bpm)_(Mean)',
    'Heart_Rate_(Median)': 'Heart_Rate_(bpm)_(Median)',
    'Heart_Rate_(Min)': 'Heart_Rate_(bpm)_(Min)',
    'Non-Invasive_BP_Diastolic_(Max)': 'Non_Invasive_Blood_Pressure_systolic_(mmHg)_(Max)',
    'Non-Invasive_BP_Diastolic_(Mean)': 'Non_Invasive_Blood_Pressure_systolic_(mmHg)_(Mean)',
    'Non-Invasive_BP_Diastolic_(Median)': 'Non_Invasive_Blood_Pressure_systolic_(mmHg)_(Median)',
    'Non-Invasive_BP_Diastolic_(Min)': 'Non_Invasive_Blood_Pressure_systolic_(mmHg)_(Min)',
    'Non-Invasive_BP_Systolic_(Max)': 'Non_Invasive_Blood_Pressure_diastolic_(mmHg)_(Max)',
    'Non-Invasive_BP_Systolic_(Mean)': 'Non_Invasive_Blood_Pressure_diastolic_(mmHg)_(Mean)',
    'Non-Invasive_BP_Systolic_(Median)': 'Non_Invasive_Blood_Pressure_diastolic_(mmHg)_(Median)',
    'Non-Invasive_BP_Systolic_(Min)': 'Non_Invasive_Blood_Pressure_diastolic_(mmHg)_(Min)',
    'Non-Invasive_BP_Mean_(Max)': 'Non_Invasive_Blood_Pressure_mean_(mmHg)_(Max)',
    'Non-Invasive_BP_Mean_(Mean)': 'Non_Invasive_Blood_Pressure_mean_(mmHg)_(Mean)',
    'Non-Invasive_BP_Mean_(Median)': 'Non_Invasive_Blood_Pressure_mean_(mmHg)_(Median)',
    'Non-Invasive_BP_Mean_(Min)': 'Non_Invasive_Blood_Pressure_mean_(mmHg)_(Min)',
    'Respiratory_Rate_(Max)': 'Respiratory_Rate_(insp/min)_(Max)',
    'Respiratory_Rate_(Mean)': 'Respiratory_Rate_(insp/min)_(Mean)',
    'Respiratory_Rate_(Median)': 'Respiratory_Rate_(insp/min)_(Median)',
    'Respiratory_Rate_(Min)': 'Respiratory_Rate_(insp/min)_(Min)',
    'O2_Saturation_(Max)': 'O2_saturation_pulseoxymetry_(%)_(Max)',
    'O2_Saturation_(Mean)': 'O2_saturation_pulseoxymetry_(%)_(Mean)',
    'O2_Saturation_(Median)': 'O2_saturation_pulseoxymetry_(%)_(Median)',
    'O2_Saturation_(Min)': 'O2_saturation_pulseoxymetry_(%)_(Min)',
    'CI_(Max)': 'Chloride_(serum)_(Max)',
    'CI_(Mean)': 'Chloride_(serum)_(Mean)',
    'CI_(Median)': 'Chloride_(serum)_(Median)',
    'CI_(Min)': 'Chloride_(serum)_(Min)',
    'calcium_(Max)': 'Calcium_non-ionized_(Max)',
    'calcium_(Mean)': 'Calcium_non-ionized_(Mean)',
    'calcium_(Median)': 'Calcium_non-ionized_(Median)',
    'calcium_(Min)': 'Calcium_non-ionized_(Min)',
    'CPK_(Max)': 'CK_(CPK)_(Max)',
    'CPK_(Mean)': 'CK_(CPK)_(Mean)',
    'CPK_(Median)': 'CK_(CPK)_(Median)',
    'CPK_(Min)': 'CK_(CPK)_(Min)',
    'Temperature_(°F)_(Max)': 'Temperature_Fahrenheit_(F)_(Max)',
    'Temperature_(°F)_(Mean)': 'Temperature_Fahrenheit_(F)_(Mean)',
    'Temperature_(°F)_(Median)': 'Temperature_Fahrenheit_(F)_(Median)',
    'Temperature_(°F)_(Min)': 'Temperature_Fahrenheit_(F)_(Min)',
    'Pain_Score_(Max)': 'Pain_Level_(Max)',
    'Pain_Score_(Mean)': 'Pain_Level_(Mean)',
    'Pain_Score_(Median)': 'Pain_Level_(Median)',
    'Pain_Score_(Min)': 'Pain_Level_(Min)',
    'LPM_O2_(Max)': 'O2_Flow_(L/min)_(Max)',
    'LPM_O2_(Mean)': 'O2_Flow_(L/min)_(Mean)',
    'LPM_O2_(Median)': 'O2_Flow_(L/min)_(Median)',
    'LPM_O2_(Min)': 'O2_Flow_(L/min)_(Min)',
    'O2_L/%_(Max)': 'Inspired_O2_Fraction_(Max)',
    'O2_L/%_(Mean)': 'Inspired_O2_Fraction_(Mean)',
    'O2_L/%_(Median)': 'Inspired_O2_Fraction_(Median)',
    'O2_L/%_(Min)': 'Inspired_O2_Fraction_(Min)',
    'ionized_calcium_(Max)': 'Ionized_Calcium_(Max)',
    'ionized_calcium_(Mean)': 'Ionized_Calcium_(Mean)',
    'ionized_calcium_(Median)': 'Ionized_Calcium_(Median)',
    'ionized_calcium_(Min)': 'Ionized_Calcium_(Min)',
    'albumin_(Max)': 'Albumin_(Max)',
    'albumin_(Mean)': 'Albumin_(Mean)',
    'albumin_(Median)': 'Albumin_(Median)',
    'albumin_(Min)': 'Albumin_(Min)',
    'GCS_Total_(Max)': 'GCS_(Max)',
    'GCS_Total_(Mean)': 'GCS_(Mean)',
    'GCS_Total_(Median)': 'GCS_(Median)',
    'GCS_Total_(Min)': 'GCS_(Min)',
    'LDH_(Max)': 'LDH_(Max)',
    'LDH_(Mean)': 'LDH_(Mean)',
    'LDH_(Median)': 'LDH_(Median)',
    'LDH_(Min)': 'LDH_(Min)',
    'ethanol_(Max)': 'ETOH_(Max)',
    'ethanol_(Mean)': 'ETOH_(Mean)',
    'ethanol_(Median)': 'ETOH_(Median)',
    'ethanol_(Min)': 'ETOH_(Min)',
    'Invasive_BP_Systolic_(Max)': 'Arterial_Blood_Pressure_systolic_(mmHg)_(Max)',
    'Invasive_BP_Systolic_(Mean)': 'Arterial_Blood_Pressure_systolic_(mmHg)_(Mean)',
    'Invasive_BP_Systolic_(Median)': 'Arterial_Blood_Pressure_systolic_(mmHg)_(Median)',
    'Invasive_BP_Systolic_(Min)': 'Arterial_Blood_Pressure_systolic_(mmHg)_(Min)',
    'Invasive_BP_Mean_(Max)': 'Arterial_Blood_Pressure_mean_(mmHg)_(Max)',
    'Invasive_BP_Mean_(Mean)': 'Arterial_Blood_Pressure_mean_(mmHg)_(Mean)',
    'Invasive_BP_Mean_(Median)': 'Arterial_Blood_Pressure_mean_(mmHg)_(Median)',
    'Invasive_BP_Mean_(Min)': 'Arterial_Blood_Pressure_mean_(mmHg)_(Min)',
    'serum_osmolality_(Max)': 'Serum_Osmolality_(Max)',
    'serum_osmolality_(Mean)': 'Serum_Osmolality_(Mean)',
    'serum_osmolality_(Median)': 'Serum_Osmolality_(Median)',
    'serum_osmolality_(Min)': 'Serum_Osmolality_(Min)',
    'troponin_-_I_(Max)': 'Troponin-T_(Max)',
    'troponin_-_I_(Mean)': 'Troponin-T_(Mean)',
    'troponin_-_I_(Median)': 'Troponin-T_(Median)',
    'troponin_-_I_(Min)': 'Troponin-T_(Min)',
    'uric_acid_(Max)': 'Uric_Acid_(Max)',
    'uric_acid_(Mean)': 'Uric_Acid_(Mean)',
    'uric_acid_(Median)': 'Uric_Acid_(Median)',
    'uric_acid_(Min)': 'Uric_Acid_(Min)',
    'ammonia_(Max)': 'Ammonia_(Max)',
    'ammonia_(Mean)': 'Ammonia_(Mean)',
    'ammonia_(Median)': 'Ammonia_(Median)',
    'ammonia_(Min)': 'Ammonia_(Min)',
    'CRP_(Max)': 'C_Reactive_Protein_(CRP)_(Max)',
    'CRP_(Mean)': 'C_Reactive_Protein_(CRP)_(Mean)',
    'CRP_(Median)': 'C_Reactive_Protein_(CRP)_(Min)',
    'CRP_(Min)': 'C_Reactive_Protein_(CRP)_(Median)',
    'fibrinogen_(Max)': 'Fibrinogen_(Max)',
    'fibrinogen_(Mean)': 'Fibrinogen_(Mean)',
    'fibrinogen_(Median)': 'Fibrinogen_(Median)',
    'fibrinogen_(Min)': 'Fibrinogen_(Min)',
    'PA_Systolic_(Max)': 'Pulmonary_Artery_Pressure_systolic_(mmHg)_(Max)',
    'PA_Systolic_(Mean)': 'Pulmonary_Artery_Pressure_systolic_(mmHg)_(Mean)',
    'PA_Systolic_(Median)': 'Pulmonary_Artery_Pressure_systolic_(mmHg)_(Median)',
    'PA_Systolic_(Min)': 'Pulmonary_Artery_Pressure_systolic_(mmHg)_(Min)',	
    'PA_Diastolic_(Max)': 'Pulmonary_Artery_Pressure_diastolic_(mmHg)_(Max)',
    'PA_Diastolic_(Mean)': 'Pulmonary_Artery_Pressure_diastolic_(mmHg)_(Mean)',
    'PA_Diastolic_(Median)': 'Pulmonary_Artery_Pressure_diastolic_(mmHg)_(Median)',
    'PA_Diastolic_(Min)': 'Pulmonary_Artery_Pressure_diastolic_(mmHg)_(Min)',
    'PA_Mean_(Max)': 'Pulmonary_Artery_Pressure_mean_(mmHg)_(Max)',
    'PA_Mean_(Mean)': 'Pulmonary_Artery_Pressure_mean_(mmHg)_(Mean)',
    'PA_Mean_(Median)': 'Pulmonary_Artery_Pressure_mean_(mmHg)_(Median)',
    'PA_Mean_(Min)': 'Pulmonary_Artery_Pressure_mean_(mmHg)_(Min)',
    'bedside_glucose_(Max)': 'Glucose_finger_stick_(range_70-100)_(Max)',
    'bedside_glucose_(Mean)': 'Glucose_finger_stick_(range_70-100)_(Mean)',
    'bedside_glucose_(Median)': 'Glucose_finger_stick_(range_70-100)_(Median)',
    'bedside_glucose_(Min)': 'Glucose_finger_stick_(range_70-100)_(Min)',
    'reticulocyte_count_(Max)': 'Reticulocyte_Count_Automated_(Mean)',
    'reticulocyte_count_(Mean)': 'Reticulocyte_Count_Automated_(Median)',
    'reticulocyte_count_(Median)': 'Reticulocyte_Count_Automated_(Min)',
    'reticulocyte_count_(Min)': 'Reticulocyte_Count_Automated_(Max)',
    '-basos_(Max)': 'Differential-Basos_(Max)',
    '-basos_(Mean)': 'Differential-Basos_(Mean)',
    '-basos_(Median)': 'Differential-Basos_(Median)',
    '-basos_(Min)': 'Differential-Basos_(Min)',
    '-eos_(Max)': 'Differential-Eos_(Max)',
    '-eos_(Mean)': 'Differential-Eos_(Mean)',
    '-eos_(Median)': 'Differential-Eos_(Median)',
    '-eos_(Min)': 'Differential-Eos_(Min)',
    '-lymphs_(Max)': 'Differential-Lymphs_(Max)',
    '-lymphs_(Mean)': 'Differential-Lymphs_(Mean)',
    '-lymphs_(Median)': 'Differential-Lymphs_(Median)',
    '-lymphs_(Min)': 'Differential-Lymphs_(Min)',
    '-monos_(Max)': 'Differential-Monos_(Max)',
    '-monos_(Mean)': 'Differential-Monos_(Mean)',
    '-monos_(Median)': 'Differential-Monos_(Median)',
    '-monos_(Min)': 'Differential-Monos_(Min)',
    '-polys_(Max)': 'Differential-Neuts_(Max)',
    '-polys_(Mean)': 'Differential-Neuts_(Mean)',
    '-polys_(Median)': 'Differential-Neuts_(Median)',
    '-polys_(Min)': 'Differential-Neuts_(Min)',
    'haptoglobin_(Max)': 'Haptoglobin_(Max)',
    'haptoglobin_(Mean)': 'Haptoglobin_(Mean)',
    'haptoglobin_(Median)': 'Haptoglobin_(Median)',
    'haptoglobin_(Min)': 'Haptoglobin_(Min)',
    'direct_bilirubin_(Max)': 'Bilirubin_Direct_(Max)',
    'direct_bilirubin_(Mean)': 'Bilirubin_Direct_(Mean)',
    'direct_bilirubin_(Median)': 'Bilirubin_Direct_(Median)',
    'direct_bilirubin_(Min)': 'Bilirubin_Direct_(Min)',
    'free_T4_(Max)': 'Thyroxine_(T4)_Free_(Max)',
    'free_T4_(Mean)': 'Thyroxine_(T4)_Free_(Mean)',
    'free_T4_(Median)': 'Thyroxine_(T4)_Free_(Median)',
    'free_T4_(Min)': 'Thyroxine_(T4)_Free_(Min)',
    'ESR_(Max)': 'Sedimentation_Rate_(Max)',
    'ESR_(Mean)': 'Sedimentation_Rate_(Mean)',
    'ESR_(Median)': 'Sedimentation_Rate_(Median)',
    'ESR_(Min)': 'Sedimentation_Rate_(Min)',
    'CPK-MB_INDEX_(Max)': 'CK-MB_(Max)',
    'CPK-MB_INDEX_(Mean)': 'CK-MB_(Mean)',
    'CPK-MB_INDEX_(Median)': 'CK-MB_(Median)',
    'CPK-MB_INDEX_(Min)': 'CK-MB_(Min)',
    'amylase_(Max)': 'Amylase_(Max)',
    'amylase_(Mean)': 'Amylase_(Mean)',
    'amylase_(Median)': 'Amylase_(Median)',
    'amylase_(Min)': 'Amylase_(Min)',
    'PEEP_(Max)': 'PEEP_set_(cmH2O)_(Max)',
    'PEEP_(Mean)': 'PEEP_set_(cmH2O)_(Mean)',
    'PEEP_(Median)': 'PEEP_set_(cmH2O)_(Median)',
    'PEEP_(Min)': 'PEEP_set_(cmH2O)_(Min)',
    'CVP_(Max)': 'Central_Venous_Pressure_(mmHg)_(Max)',
    'CVP_(Mean)': 'Central_Venous_Pressure_(mmHg)_(Mean)',
    'CVP_(Median)': 'Central_Venous_Pressure_(mmHg)_(Median)',
    'CVP_(Min)': 'Central_Venous_Pressure_(mmHg)_(Min)',
    'total_bilirubin_(Max)': 'Total_Bilirubin_(Max)',
    'total_bilirubin_(Mean)': 'Total_Bilirubin_(Mean)',
    'total_bilirubin_(Median)': 'Total_Bilirubin_(Median)',
    'total_bilirubin_(Min)': 'Total_Bilirubin_(Min)',
    'Invasive_BP_Diastolic_(Max)': 'Arterial_Blood_Pressure_diastolic_(mmHg)_(Max)',
    'Invasive_BP_Diastolic_(Mean)': 'Arterial_Blood_Pressure_diastolic_(mmHg)_(Mean)',
    'Invasive_BP_Diastolic_(Median)': 'Arterial_Blood_Pressure_diastolic_(mmHg)_(Median)',
    'Invasive_BP_Diastolic_(Min)': 'Arterial_Blood_Pressure_diastolic_(mmHg)_(Min)',
    'unitdischargestatus': 'hospital_expire_flag',
    'LOS': 'los'
}

# Replace the DataFrame and column names mapping
eicu_temp.rename(columns=column_eicu_mapping, inplace=True)



column_mimic_mapping = {
    'Temperature Fahrenheit (°F)': 'Temperature Fahrenheit (F)',
}

# Replace the DataFrame and column names mapping
df_mimic_unique.rename(columns=column_mimic_mapping, inplace=True)

# Remove "-" from the 'subject_id' column in eicu
eicu_temp['subject_id'] = eicu_temp['subject_id'].str.replace('-', '')

# Convert 'subject_id' in eicu to int64
eicu_temp['subject_id'] = eicu_temp['subject_id'].astype(np.int64)

C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\3774024962.py:323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eicu_temp.rename(columns=column_eicu_mapping, inplace=True)
C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\3774024962.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mimic_unique.rename(columns=column_mimic_mapping, inplace=True)
C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\3774024962.py:335: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [ ]:
# Remove duplicate columns
df_mimic_unique = df_mimic_unique.loc[:, ~df_mimic_unique.columns.duplicated()]

# Remove duplicate columns
eicu_temp = eicu_temp.loc[:, ~eicu_temp.columns.duplicated()]

In [38]:
display(df_mimic_unique)

display(eicu_temp)

,row_count,subject_id,hadm_id,Time_Zone,gender,age,race,Alanine_Aminotransferase_(ALT)_(Max),Alanine_Aminotransferase_(ALT)_(Mean),Alanine_Aminotransferase_(ALT)_(Median),...,hospital_expire_flag,los,Glucose_(Max),Glucose_(Mean),Glucose_(Median),Glucose_(Min),pH_(Max),pH_(Mean),pH_(Median),pH_(Min)
0,1,10004733,27411876,1,M,51,UNKNOWN,46.0,46.0,46.0,...,Survive,8.357373,86.0,86.0,86.0,86.0,6.715,6.715,6.715,6.715
1,2,10004733,27411876,2,M,51,UNKNOWN,46.0,46.0,46.0,...,Survive,8.357373,86.0,86.0,86.0,86.0,6.715,6.715,6.715,6.715
2,3,10004733,27411876,3,M,51,UNKNOWN,46.0,46.0,46.0,...,Survive,8.357373,86.0,86.0,86.0,86.0,6.715,6.715,6.715,6.715
3,4,10004733,27411876,4,M,51,UNKNOWN,46.0,46.0,46.0,...,Survive,8.357373,94.0,90.0,90.0,86.0,6.715,6.715,6.715,6.715
4,5,10004733,27411876,5,M,51,UNKNOWN,46.0,46.0,46.0,...,Survive,8.357373,94.0,90.0,90.0,86.0,6.715,6.715,6.715,6.715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58139,58140,19999987,23865745,12,F,57,UNKNOWN,63.0,63.0,63.0,...,Survive,1.937847,113.0,113.0,113.0,113.0,6.445,6.445,6.445,6.445
58140,58141,19999987,23865745,13,F,57,UNKNOWN,63.0,63.0,63.0,...,Survive,1.937847,113.0,113.0,113.0,113.0,6.445,6.445,6.445,6.445
58141,58142,19999987,23865745,14,F,57,UNKNOWN,63.0,63.0,63.0,...,Survive,1.937847,113.0,113.0,113.0,113.0,6.445,6.445,6.445,6.445
58142,58143,19999987,23865745,15,F,57,UNKNOWN,63.0,63.0,63.0,...,Survive,1.937847,113.0,113.0,113.0,113.0,6.445,6.445,6.445,6.445


,row_count,subject_id,hadm_id,Time_Zone,gender,age,race,Albumin_(Max),Albumin_(Mean),Albumin_(Median),...,White_Blood_Cells_(Max),White_Blood_Cells_(Mean),White_Blood_Cells_(Median),White_Blood_Cells_(Min),hospital_expire_flag,los,Glucose_finger_stick_(range_70-100)_(Max),Glucose_finger_stick_(range_70-100)_(Mean),Glucose_finger_stick_(range_70-100)_(Median),Glucose_finger_stick_(range_70-100)_(Min)
0,1,233029,142974,1,F,54,BLACK/AFRICAN AMERICAN,NaN,NaN,NaN,...,11.1,11.100000,11.10,11.1,0,2.72,103.0,103.000000,103.0,103.0
1,2,233029,142974,2,F,54,BLACK/AFRICAN AMERICAN,NaN,NaN,NaN,...,11.1,11.100000,11.10,11.1,0,2.72,103.0,101.500000,101.5,100.0
2,3,233029,142974,3,F,54,BLACK/AFRICAN AMERICAN,NaN,NaN,NaN,...,11.1,11.100000,11.10,11.1,0,2.72,103.0,103.000000,103.0,103.0
3,4,233029,142974,4,F,54,BLACK/AFRICAN AMERICAN,NaN,NaN,NaN,...,11.1,11.100000,11.10,11.1,0,2.72,126.0,109.666667,103.0,100.0
4,5,233029,142974,5,F,54,BLACK/AFRICAN AMERICAN,NaN,NaN,NaN,...,11.1,11.100000,11.10,11.1,0,2.72,103.0,103.000000,103.0,103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86267,86268,3520783,3353144,12,M,51,BLACK/AFRICAN AMERICAN,2.6,2.6,2.6,...,18.1,18.100000,18.10,18.1,0,11.05,192.0,192.000000,192.0,192.0
86268,86269,3520783,3353144,13,M,51,BLACK/AFRICAN AMERICAN,2.6,2.6,2.6,...,23.2,20.433333,20.00,18.1,0,11.05,192.0,162.666667,157.5,145.0
86269,86270,3520783,3353144,14,M,51,BLACK/AFRICAN AMERICAN,2.6,2.6,2.6,...,23.2,20.650000,20.65,18.1,0,11.05,192.0,168.000000,167.0,145.0
86270,86271,3520783,3353144,15,M,51,BLACK/AFRICAN AMERICAN,2.6,2.6,2.6,...,18.1,18.100000,18.10,18.1,0,11.05,192.0,161.571429,155.0,145.0


In [42]:
# Replace 'Survive' with 0 and 'Death' with 1 in the 'hospital_expire_flag' column
df_mimic_unique['hospital_expire_flag'] = df_mimic_unique['hospital_expire_flag'].replace({'Survive': 0, 'Death': 1})

C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\2443844813.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_mimic_unique['hospital_expire_flag'] = df_mimic_unique['hospital_expire_flag'].replace({'Survive': 0, 'Death': 1})
C:\Users\Dimopoulos\AppData\Local\Temp\ipykernel_13404\2443844813.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mimic_unique['hospital_expire_flag'] = df_mimic_unique['hospital_expire_flag'].replace({'Survive': 0, 'Death': 1})


In [48]:
#Export Columns
column_names = eicu_temp.columns

# Create a dataframe from the column names
columns_df = pd.DataFrame(column_names, columns=['Column_Name'])

# Export to CSV
columns_df.to_csv('eicu_names.csv', index=False)

In [44]:
# Check if mimic and eicu datasets have the same dtype and header names

# Get the column names and dtypes of mimic_df
mimic_info = df_mimic_unique.dtypes

# Get the column names and dtypes of eicu_df
eicu_info = eicu_temp.dtypes

# Check if the column names are the same
if all(mimic_info.index == eicu_info.index):
    print("The column names are the same.")
else:
    print("The column names are different.")

print("\n")

# Check if the number of columns is the same
if len(mimic_info) != len(eicu_info):
    print("Number of columns is different between mimic_df and eicu_df.")
else:
    # Iterate over the columns and compare the data type.
    for column_name in mimic_info.index:
        mimic_dtype = mimic_info[column_name]
        eicu_dtype = eicu_info[column_name]
        if mimic_dtype != eicu_dtype:
            print(f"Column '{column_name}' has different data types: mimic_df has '{mimic_dtype}' and eicu_df has '{eicu_dtype}'.")

The column names are different.


Column 'age' has different data types: mimic_df has 'int64' and eicu_df has 'object'.


KeyError: 'Alkaline_Phosphatase_(Mean)'

In [ ]:
# Export the merged DataFrame to a CSV file
mimic_temp.to_csv('CSV\\exports\\final\\mimic_mean_final.csv', index=False)

In [ ]:
# Export the merged DataFrame to a CSV file
eicu_temp.to_csv('CSV\\exports\\final\\eicu_mean_final.csv', index=False)